In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
test_path=""
img_path=""
mask_path=""
out_path=""
check_path=""


In [ ]:
import numpy as np
import cv2
from queue import Queue

def process_mask(mask):
    height, width = mask.shape
    visited = np.zeros_like(mask, dtype=bool)
    result_mask = np.zeros_like(mask)
    Q = Queue()
    for x in range(height):
        for y in range(width):
            if (x == 0 or y == 0 or x == height - 1 or y == width - 1) :
              if(mask[x][y] == 255):
                Q.put((x, y))
                visited[x][y] = True


    dirs = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    while not Q.empty():
        x, y = Q.get()
        result_mask[x][y] = 1
        for dx, dy in dirs:
            nx, ny = x + dx, y + dy
            if nx < 0 or nx >= height or ny < 0 or ny >= width:
                continue
            if mask[nx][ny] == 255 and not visited[nx][ny]:
                Q.put((nx, ny))
                visited[nx][ny] = True
    return result_mask

In [ ]:
# enviroment generation
import os
import subprocess
import PIL.Image as Image
import cv2
import numpy as np
import torch
from torchvision import transforms
import torch.nn.functional as F
input_path=""
mask_path=""
out_path=""

i=0
for name in os.listdir(mask_path):
  input_image=os.path.join(input_path,name)
  input_mask=os.path.join(mask_path,name)
  img=cv2.imread(input_mask, 0)
  img=torch.Tensor(img)
  img=process_mask(img)
  size=img.shape
  img=torch.Tensor(img)
  padding = (50, 50, 50, 50)
  img_padded = F.pad(img, padding, "constant", 1)
  img_padded=img_padded.numpy()
  numpy_array = (img_padded*255).astype(np.uint8)
  pil_image = Image.fromarray(numpy_array,'L')
  pil_image=pil_image.resize(size)
  pil_image.save('aug_mask.png')
  input_mask='aug_mask.png'
  in_img=cv2.imread(input_image, cv2.IMREAD_COLOR)
  padding = (50, 50, 50, 50)
  in_img=torch.Tensor(in_img)
  in_img=torch.permute(in_img,(2,0,1))
  in_img = F.pad(in_img, padding, "constant", 0)
  in_img=torch.permute(in_img,(1,2,0))
  in_img=in_img.numpy()
  resized_input=cv2.resize(in_img,size)
  resized_input=resized_input[:, :, :3]
  img_bgr = cv2.cvtColor(resized_input, cv2.COLOR_RGB2BGR)
  cv2.imwrite("input_image.png",img_bgr)
  input_image="input_image.png"
  out_name=os.path.join(out_path,name)
  params={"--mask":input_mask,"--checkpoint":check_path,"--image":input_image,"--out":out_name}
  params_list = []
  for key, value in params.items():
    params_list.append(key)
    params_list.append(value)

# 把主命令和参数放在一个列表中
  command = ['python3', test_path] + params_list

# 在subprocess.run中运行
  subprocess.run(command)
  print(out_name)
  i+=1



In [ ]:
# building erase
import os
import subprocess
import PIL.Image as Image
import cv2
import numpy as np
#import torch
input_path=""
mask_path=""
out_path=""
if not os.path.exists(out_path):
  os.mkdir(out_path)
i=0
for name in os.listdir(mask_path):
  input_image=os.path.join(input_path,name)
  input_mask=os.path.join(mask_path,name)
  img=cv2.imread(input_mask, 0)
  img=img.astype(np.uint8)
  kernel = np.ones((3,3),np.uint8)
  dilation = cv2.dilate(img,kernel,iterations = 15)
  cv2.imwrite('dilation.png', dilation)
  input_mask='dilation.png'
  out_name=os.path.join(out_path,name)
  params={"--mask":input_mask,"--checkpoint":check_path,"--image":input_image,"--out":out_name}
  params_list = []
  for key, value in params.items():
    params_list.append(key)
    params_list.append(value)
  command = ['python3', test_path] + params_list
  subprocess.run(command)
  print(i)
  i+=1